# <font color='red' style='font-size: 30px;'> Rede Complexa Baseada em CDR's </font>
<hr style='border: 2px solid red;'>

## <font color = 'black' style='font-size: 26px;'> Imports </font>  
<hr style = 'border: 1.5px solid black;'>

In [3]:
# Lista de Imports
import pandas as pd
import numpy as np
import networkx as nx
from functions import save_with_pickle, load_with_pickle

# Variáveis Globais
EXECUTAR = True

## <font color = 'black' style='font-size: 26px;'> Importando os Dados </font>  
<hr style = 'border: 1.5px solid black;'>

In [5]:
# Importação dos Dados
dataset = pd.read_csv('../Dados/cdr_processed_Uberlândia.csv', sep=',')

## <font color = 'black' style='font-size: 26px;'> Realizando o Pré-Processamento </font>  
<hr style = 'border: 1.5px solid black;'>

In [7]:
# Selecionando as ligações com duração maiores que 15 segundos
df = dataset[dataset['Duração'] >= 0.24]
df.shape

(7618927, 6)

In [8]:
len(df['Data'].unique())

30

In [9]:
df = df.groupby(['ID_Origem', 'ID_Destino']).filter(lambda x: len(x) >= 5)

In [10]:
agrup_origem = df.groupby(by='ID_Origem').filter(lambda x: len(x) <= 200)['ID_Origem']
agrup_origem.value_counts()

ID_Origem
B2601CCF8ECC76E3BB1C891F4D28C5CE    200
B22F9C56D2E5AB59D613EE5FC8763F9A    200
8B66A139CB2F2B08D1727D44C10EDDBA    200
F60AEA943E44C5C252D28BCF160B65DC    200
78366FC1DB0F774F313A7C7D80283F6C    200
                                   ... 
33D67FFC9A6776899DB36961D412D95E      5
0E137186B4EA9142FE278D5E7A82E3AE      5
27FB14416E2296CCF3222B0F5B736111      5
1527449E0CE5A9B1A3FCF691452961F7      5
3C1ACC5E015475DE21FDF1B95E1B9361      5
Name: count, Length: 153547, dtype: int64

In [11]:
agrup_destino = df.groupby(by='ID_Destino').filter(lambda x: len(x) <= 200)['ID_Destino']
agrup_destino.value_counts()

ID_Destino
09D43014CAAAF12A3ACF83AD4DF6F8C2    200
2BDFCA39518BFAE070908E54C0F8B4DF    200
B9F6C3DBE1D3904D8EA498720C731352    200
4BD86B6C8C715105D153D89B625EB932    200
4D8429AC3DCF16B1599822470359909A    200
                                   ... 
BB3A2509DB9F8479AB0F8B13BE933BA7      5
19601781FF5CC6493DF4AC2165668E0C      5
E2BC55280D7613A9CA20084D0DA58899      5
A76A5B1E5F25691132153074AF8559C7      5
9C481C23251264799C3637AE478DE70C      5
Name: count, Length: 322422, dtype: int64

In [12]:
# Todos os IDs que fizeram ou receberam mais de 6 ligações
nos = np.unique(np.append(agrup_origem, agrup_destino))
save_with_pickle(nos, '../Dados/Processados/', 'nos')
len(nos)

372045

In [13]:
df = df.query("ID_Origem in @nos | ID_Destino in @nos")
df.shape

(7443440, 6)

In [ ]:
if EXECUTAR:    
    # Instanciando o dicionário
    users = {}
    for ID in df['ID_Origem'].unique():
        users[ID] = {'Ligações': list(), 'Data': list(), 'Hora': list(), 'Duração': list(), 'Antena': list()}
        
    # Agrupando as ligações em um dicionário    
    for _, row in df.iterrows():
        users[row['ID_Origem']]['Ligações'].append(row['ID_Destino'])
        users[row['ID_Origem']]['Data'].append(row['Data'])
        users[row['ID_Origem']]['Hora'].append(row['Hora'])
        users[row['ID_Origem']]['Duração'].append(row['Duração'])
        users[row['ID_Origem']]['Antena'].append(row['Antena'])
    
    # Salva o dicionário em um objeto pickle
    save_with_pickle(users, '../Dados/Processados/', 'dict_dados')
    
else:
    # Carrega o dicionário em um objeto pickle
    users = load_with_pickle('../Dados/Processados/', 'dict_dados')

## <font color = 'black' style='font-size: 26px;'> Criando a Rede </font>  
<hr style = 'border: 1.5px solid black;'>

In [ ]:
def contabiliza_ligacoes(origem, destino, dicionario):
    total = 0
    if origem in dicionario:
        total += dicionario[origem]['Ligações'].count(destino)
        
    if destino in dicionario:
        total += dicionario[destino]['Ligações'].count(origem)
        
    return total    

In [ ]:
def inicializa_grafo(nos):
    grafo = nx.Graph() 
    for i in nos:
        grafo.add_node(i)
    return grafo

In [ ]:
def cria_arestas(grafo, nos, dicionario):
    
    # Cria arestas entre todos os nós da lista
    for i in dicionario.keys():
        for j in dicionario[i]['Ligações']:
            if not grafo.has_edge(i, j):
                grafo.add_edge(i, j)
    return grafo

In [ ]:
def gera_grafo(nos, dicionario):    
    grafo = inicializa_grafo(nos)
    grafo = cria_arestas(grafo, nos, dicionario)
    return grafo

In [ ]:
##### grafo = gera_grafo(nos, users)
# nx.write_graphml(grafo, "cdr_grafo.graphml")